<a href="https://colab.research.google.com/github/harrymkwn/InfluenceAnalysis/blob/master/AllEkphrasisMod.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import numpy as np
import tensorflow as tf
from gensim.models import Word2Vec
import matplotlib.pyplot as plt
from sklearn.linear_model import LogisticRegression
from sklearn.multiclass import OneVsRestClassifier
from gensim.models import KeyedVectors
from sklearn.model_selection import train_test_split
from google.colab import  drive
drive.mount('/content/drive')
import nltk
from nltk.corpus import stopwords
nltk.download('stopwords')
device_name = tf.test.gpu_device_name()
print(device_name)
df_train = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/train_clean.csv',names=['Tweet','Emotion'])
df_test = pd.read_csv('/content/drive/MyDrive/InfluenceAnalysis/Newdata/test_clean.csv',names=['Tweet','Emotion'])
df_train = df_train.sample(frac=1)
df_test = df_train.sample(frac=1)

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
/device:GPU:0


In [ ]:
df_train.head(1000)

,Tweet,Emotion
3146,low low low low low low low low low,2
12425,crpitt i am send healthi heal thought in the m...,1
10443,run errand with geeta feel like i have the plagu,2
2417,breakfastnt wa not much of a debat,0
1436,dstroyr rockdamullet aww i have no internet ri...,1
...,...,...
9610,dmann all those link on aicn are broken did th...,0
10874,she gone i miss my kay way alreadi,2
468,crunchmonkey well i just got home and it not o...,0
1979,ok so i did shed a few tear watch the preview ...,1


In [ ]:
df_test.Emotion.unique()

array(['2', '0', '1', '3', 'emotion'], dtype=object)

In [ ]:
df_train.nunique()

Tweet      12370
Emotion        5
dtype: int64

In [ ]:
category_dict = {'0':[1,0,0,0],'1':[0,1,0,0],'2':[0,0,1,0],'3':[0,0,0,1],'emotion':[1,0,0,0]}
train_data_tweet = [x.lower().split() for x in df_train['Tweet']]
train_data_cat = np.array([category_dict[x] for x in df_train['Emotion']])
test_data_tweet = [x.lower().split() for x in df_test['Tweet']]
test_data_cat = np.array([category_dict[x] for x in df_test['Emotion']])

data_tweet = train_data_tweet + test_data_tweet

In [ ]:
# Parameters
Min_count = 0
Embedding_size = 100
Window_size = 5
Negative_sampling = 00

In [ ]:
w2v_sg = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=1)
w2v_sg.build_vocab(data_tweet)
w2v_sg.train(data_tweet, total_examples=w2v_sg.corpus_count, epochs=5)
w2v_cbow = Word2Vec(min_count=Min_count,
                     window=Window_size,
                     size=Embedding_size,
                     negative=Negative_sampling,sg=0)
w2v_cbow.build_vocab(data_tweet)
w2v_cbow.train(data_tweet, total_examples=w2v_cbow.corpus_count, epochs=5)

(1303468, 1724630)

In [ ]:
vocab_sg = w2v_sg.wv.vocab
vocab_sg = [x for x in vocab_sg]
vocab_cbow = w2v_cbow.wv.vocab
vocab_cbow = [x for x in vocab_cbow] 
vocab_glove = {}
with open("/content/drive/My Drive/InfluenceAnalysis/glove/glove.twitter.27B.100d.txt", 'r', encoding="utf-8") as f:
  for line in f:
      values = line.split()
      word = values[0]
      vector = np.asarray(values[1:], "float32")
      vocab_glove[word] = vector

In [ ]:
batch_size = 500
Max_input_size = max([len(x) for x in data_tweet])
mul_factor=1.5

In [ ]:
X_train = train_data_tweet
X_test = test_data_tweet
y_train = train_data_cat
y_test = test_data_cat 
result_table = []

In [ ]:
def vec_gen(w2v,vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([w2v.wv.get_vector(i)*mul_factor if (i in vocab and (nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='JJ' or nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='NN')) else w2v.wv.get_vector(i) for i in x if i in vocab]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

def glove_gen(vocab,data_tweet,data_cat,batch_size,Max_input_size,Embedding_size,mul_factor):
  while True:
    for k in range(int(len(data_tweet)/batch_size)):
      temp = np.array([np.array([np.array(vocab[i])*mul_factor if (i in vocab.keys() and (nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='JJ' or nltk.pos_tag(nltk.word_tokenize(i))[0][1]=='NN')) else np.array(vocab[i]) for i in x if i in vocab.keys()]) for x in data_tweet[k*(batch_size):(k+1)*(batch_size)]])
      temp = np.array([np.pad(x.flatten(),(0,Max_input_size*Embedding_size-len(x.flatten()))).reshape(Max_input_size,Embedding_size) for x in temp])
      tempres = data_cat[k*(batch_size):(k+1)*(batch_size)]
      yield (temp,tempres)

In [ ]:
epochs = 20

In [ ]:
model_sg = tf.keras.Sequential()
model_sg.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg.add(tf.keras.layers.LSTM(50))
model_sg.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_LSTM")


Instructions for updating:
Please use Model.fit, which supports generators.
Epoch 1/20
24/24 [==============================] - 5s 192ms/step - loss: 1.2480 - accuracy: 0.5724
Epoch 2/20
24/24 [==============================] - 5s 194ms/step - loss: 1.0160 - accuracy: 0.5857
Epoch 3/20
24/24 [==============================] - 5s 193ms/step - loss: 0.9857 - accuracy: 0.5857
Epoch 4/20
24/24 [==============================] - 5s 195ms/step - loss: 0.9846 - accuracy: 0.5857
Epoch 5/20
24/24 [==============================] - 5s 196ms/step - loss: 0.9847 - accuracy: 0.5857
Epoch 6/20
24/24 [==============================] - 5s 196ms/step - loss: 0.9846 - accuracy: 0.5857
Epoch 7/20
24/24 [==============================] - 5s 195ms/step - loss: 0.9846 - accuracy: 0.5857
Epoch 8/20
24/24 [==============================] - 5s 206ms/step - loss: 0.9846 - accuracy: 0.5857
Epoch 9/20
24/24 [==============================] - 5s 195ms/step - loss: 0.9846 - accuracy: 0.5857
Epoch 10/20
24/24 [=====

In [ ]:
model_sg_bi = tf.keras.Sequential()
model_sg_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_sg_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_bi.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_bi.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_BI")

Epoch 1/20
24/24 [==============================] - 5s 194ms/step - loss: 1.2091 - accuracy: 0.5756
Epoch 2/20
24/24 [==============================] - 5s 197ms/step - loss: 1.0088 - accuracy: 0.5857
Epoch 3/20
24/24 [==============================] - 5s 195ms/step - loss: 0.9841 - accuracy: 0.5857
Epoch 4/20
24/24 [==============================] - 5s 196ms/step - loss: 0.9843 - accuracy: 0.5857
Epoch 5/20
24/24 [==============================] - 5s 196ms/step - loss: 0.9843 - accuracy: 0.5857
Epoch 6/20
24/24 [==============================] - 5s 196ms/step - loss: 0.9841 - accuracy: 0.5857
Epoch 7/20
24/24 [==============================] - 5s 199ms/step - loss: 0.9840 - accuracy: 0.5857
Epoch 8/20
24/24 [==============================] - 5s 194ms/step - loss: 0.9840 - accuracy: 0.5857
Epoch 9/20
24/24 [==============================] - 5s 197ms/step - loss: 0.9839 - accuracy: 0.5857
Epoch 10/20
24/24 [==============================] - 5s 196ms/step - loss: 0.9838 - accuracy: 0.5857

In [ ]:
model_sg_rnn = tf.keras.Sequential()
model_sg_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_sg_rnn.add(tf.keras.layers.SimpleRNN(50))
model_sg_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_sg_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_sg_rnn.fit_generator(generator = vec_gen(w2v_sg,vocab_sg,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_sg_rnn.evaluate(x = vec_gen(w2v_sg,vocab_sg,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("SG_RNN")

Epoch 1/20
24/24 [==============================] - 5s 205ms/step - loss: 1.0649 - accuracy: 0.5742
Epoch 2/20
24/24 [==============================] - 5s 206ms/step - loss: 0.9849 - accuracy: 0.5857
Epoch 3/20
24/24 [==============================] - 5s 204ms/step - loss: 0.9843 - accuracy: 0.5857
Epoch 4/20
24/24 [==============================] - 5s 206ms/step - loss: 0.9842 - accuracy: 0.5857
Epoch 5/20
24/24 [==============================] - 5s 209ms/step - loss: 0.9842 - accuracy: 0.5857
Epoch 6/20
24/24 [==============================] - 5s 207ms/step - loss: 0.9842 - accuracy: 0.5857
Epoch 7/20
24/24 [==============================] - 5s 204ms/step - loss: 0.9842 - accuracy: 0.5857
Epoch 8/20
24/24 [==============================] - 5s 207ms/step - loss: 0.9842 - accuracy: 0.5857
Epoch 9/20
24/24 [==============================] - 5s 205ms/step - loss: 0.9842 - accuracy: 0.5857
Epoch 10/20
24/24 [==============================] - 5s 205ms/step - loss: 0.9842 - accuracy: 0.5857

In [ ]:
model_cbow = tf.keras.Sequential()
model_cbow.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow.add(tf.keras.layers.LSTM(50))
model_cbow.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("CBOW_LSTM")

Epoch 1/20
24/24 [==============================] - 5s 191ms/step - loss: 1.2410 - accuracy: 0.5730
Epoch 2/20
24/24 [==============================] - 5s 191ms/step - loss: 1.0213 - accuracy: 0.5857
Epoch 3/20
24/24 [==============================] - 5s 195ms/step - loss: 0.9851 - accuracy: 0.5857
Epoch 4/20
24/24 [==============================] - 5s 189ms/step - loss: 0.9844 - accuracy: 0.5857
Epoch 5/20
24/24 [==============================] - 5s 190ms/step - loss: 0.9844 - accuracy: 0.5857
Epoch 6/20
24/24 [==============================] - 5s 195ms/step - loss: 0.9845 - accuracy: 0.5857
Epoch 7/20
24/24 [==============================] - 5s 205ms/step - loss: 0.9844 - accuracy: 0.5857
Epoch 8/20
24/24 [==============================] - 5s 189ms/step - loss: 0.9844 - accuracy: 0.5857
Epoch 9/20
24/24 [==============================] - 5s 193ms/step - loss: 0.9844 - accuracy: 0.5857
Epoch 10/20
24/24 [==============================] - 5s 192ms/step - loss: 0.9844 - accuracy: 0.5857

In [ ]:
model_cbow_bi = tf.keras.Sequential()
model_cbow_bi.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_cbow_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_bi.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_bi.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("CBOW_BI")

Epoch 1/20
24/24 [==============================] - 5s 191ms/step - loss: 1.2255 - accuracy: 0.5711
Epoch 2/20
24/24 [==============================] - 5s 199ms/step - loss: 1.0110 - accuracy: 0.5857
Epoch 3/20
24/24 [==============================] - 5s 198ms/step - loss: 0.9840 - accuracy: 0.5857
Epoch 4/20
24/24 [==============================] - 5s 199ms/step - loss: 0.9844 - accuracy: 0.5857
Epoch 5/20
24/24 [==============================] - 5s 195ms/step - loss: 0.9844 - accuracy: 0.5857
Epoch 6/20
24/24 [==============================] - 5s 201ms/step - loss: 0.9844 - accuracy: 0.5857
Epoch 7/20
24/24 [==============================] - 5s 195ms/step - loss: 0.9843 - accuracy: 0.5857
Epoch 8/20
24/24 [==============================] - 5s 194ms/step - loss: 0.9842 - accuracy: 0.5857
Epoch 9/20
24/24 [==============================] - 5s 197ms/step - loss: 0.9840 - accuracy: 0.5857
Epoch 10/20
24/24 [==============================] - 5s 200ms/step - loss: 0.9839 - accuracy: 0.5857

In [ ]:
model_cbow_rnn = tf.keras.Sequential()
model_cbow_rnn.add(tf.keras.Input(shape=(Max_input_size,Embedding_size)))
model_cbow_rnn.add(tf.keras.layers.SimpleRNN(50))
model_cbow_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_cbow_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_cbow_rnn.fit_generator(generator = vec_gen(w2v_cbow,vocab_cbow,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_cbow_rnn.evaluate(x = vec_gen(w2v_cbow,vocab_cbow,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("CBOW_RNN")

Epoch 1/20
24/24 [==============================] - 5s 210ms/step - loss: 1.0961 - accuracy: 0.4832
Epoch 2/20
24/24 [==============================] - 5s 209ms/step - loss: 0.9867 - accuracy: 0.5857
Epoch 3/20
24/24 [==============================] - 5s 209ms/step - loss: 0.9856 - accuracy: 0.5857
Epoch 4/20
24/24 [==============================] - 5s 210ms/step - loss: 0.9854 - accuracy: 0.5857
Epoch 5/20
24/24 [==============================] - 5s 210ms/step - loss: 0.9853 - accuracy: 0.5857
Epoch 6/20
24/24 [==============================] - 5s 206ms/step - loss: 0.9853 - accuracy: 0.5857
Epoch 7/20
24/24 [==============================] - 5s 206ms/step - loss: 0.9853 - accuracy: 0.5857
Epoch 8/20
24/24 [==============================] - 5s 205ms/step - loss: 0.9853 - accuracy: 0.5857
Epoch 9/20
24/24 [==============================] - 5s 210ms/step - loss: 0.9853 - accuracy: 0.5857
Epoch 10/20
24/24 [==============================] - 5s 206ms/step - loss: 0.9852 - accuracy: 0.5857

In [ ]:
epochs=40
Embedding_size=100

In [ ]:
model_glove = tf.keras.Sequential()
model_glove.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove.add(tf.keras.layers.LSTM(50))
model_glove.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("GLOVE_LSTM")


Epoch 1/40
24/24 [==============================] - 1s 31ms/step - loss: 1.1710 - accuracy: 0.5789
Epoch 2/40
24/24 [==============================] - 1s 31ms/step - loss: 0.9860 - accuracy: 0.5857
Epoch 3/40
24/24 [==============================] - 1s 32ms/step - loss: 0.9754 - accuracy: 0.5857
Epoch 4/40
24/24 [==============================] - 1s 32ms/step - loss: 0.9698 - accuracy: 0.5857
Epoch 5/40
24/24 [==============================] - 1s 31ms/step - loss: 0.9670 - accuracy: 0.5859
Epoch 6/40
24/24 [==============================] - 1s 32ms/step - loss: 0.9617 - accuracy: 0.5913
Epoch 7/40
24/24 [==============================] - 1s 33ms/step - loss: 0.9466 - accuracy: 0.5933
Epoch 8/40
24/24 [==============================] - 1s 31ms/step - loss: 0.9192 - accuracy: 0.6037
Epoch 9/40
24/24 [==============================] - 1s 30ms/step - loss: 0.9039 - accuracy: 0.6095
Epoch 10/40
24/24 [==============================] - 1s 31ms/step - loss: 0.8965 - accuracy: 0.6109
Epoch 11/

In [ ]:
model_glove_bi = tf.keras.Sequential()
model_glove_bi.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_bi.add(tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(50),merge_mode='concat'))
model_glove_bi.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_bi.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_bi.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_bi.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("GLOVE_BI")

Epoch 1/40
24/24 [==============================] - 1s 31ms/step - loss: 1.0463 - accuracy: 0.5863
Epoch 2/40
24/24 [==============================] - 1s 32ms/step - loss: 0.9498 - accuracy: 0.5909
Epoch 3/40
24/24 [==============================] - 1s 35ms/step - loss: 0.9296 - accuracy: 0.5975
Epoch 4/40
24/24 [==============================] - 1s 37ms/step - loss: 0.9116 - accuracy: 0.6016
Epoch 5/40
24/24 [==============================] - 1s 34ms/step - loss: 0.8992 - accuracy: 0.6085
Epoch 6/40
24/24 [==============================] - 1s 32ms/step - loss: 0.8893 - accuracy: 0.6124
Epoch 7/40
24/24 [==============================] - 1s 33ms/step - loss: 0.8822 - accuracy: 0.6144
Epoch 8/40
24/24 [==============================] - 1s 32ms/step - loss: 0.8752 - accuracy: 0.6174
Epoch 9/40
24/24 [==============================] - 1s 34ms/step - loss: 0.8685 - accuracy: 0.6205
Epoch 10/40
24/24 [==============================] - 1s 33ms/step - loss: 0.8621 - accuracy: 0.6227
Epoch 11/

In [ ]:
model_glove_rnn = tf.keras.Sequential()
model_glove_rnn.add(tf.keras.Input(shape=(Max_input_size,100)))
model_glove_rnn.add(tf.keras.layers.SimpleRNN(50))
model_glove_rnn.add(tf.keras.layers.Dense(4, activation='softmax'))
model_glove_rnn.compile(loss='categorical_crossentropy',optimizer='adam', 
                           metrics=['accuracy'])
history = model_glove_rnn.fit_generator(generator = glove_gen(vocab_glove,X_train,y_train,batch_size,Max_input_size,Embedding_size,mul_factor),steps_per_epoch=int(len(X_train)/batch_size),epochs=epochs)
history = model_glove_rnn.evaluate(x = glove_gen(vocab_glove,X_test,y_test,batch_size,Max_input_size,Embedding_size,mul_factor),steps=int(len(y_test)/batch_size))
print(history)

result_table+=[history[1]]
print("GLOVE_RNN")

Epoch 1/40
24/24 [==============================] - 1s 44ms/step - loss: 1.0830 - accuracy: 0.5505
Epoch 2/40
24/24 [==============================] - 1s 48ms/step - loss: 0.9875 - accuracy: 0.5856
Epoch 3/40
24/24 [==============================] - 1s 45ms/step - loss: 0.9869 - accuracy: 0.5856
Epoch 4/40
24/24 [==============================] - 1s 45ms/step - loss: 0.9856 - accuracy: 0.5856
Epoch 5/40
24/24 [==============================] - 1s 49ms/step - loss: 0.9856 - accuracy: 0.5856
Epoch 6/40
24/24 [==============================] - 1s 44ms/step - loss: 0.9856 - accuracy: 0.5856
Epoch 7/40
24/24 [==============================] - 1s 43ms/step - loss: 0.9855 - accuracy: 0.5856
Epoch 8/40
24/24 [==============================] - 1s 47ms/step - loss: 0.9854 - accuracy: 0.5857
Epoch 9/40
24/24 [==============================] - 1s 43ms/step - loss: 0.9853 - accuracy: 0.5857
Epoch 10/40
24/24 [==============================] - 1s 44ms/step - loss: 0.9851 - accuracy: 0.5857
Epoch 11/

In [ ]:
print("\tLstm\t\t\tBiLstm\t\t\tRnn")
print("sg",result_table[:3])
print("cbow",result_table[3:6])
print("glove",result_table[6:9])

	Lstm			BiLstm			Rnn
sg [0.5865833163261414, 0.5865833163261414, 0.5865833163261414]
cbow [0.5865833163261414, 0.5865833163261414, 0.5865833163261414]
glove [0.6365833282470703, 0.6725833415985107, 0.5865833163261414]
